## Practical Work 3
### Session 1: Machine Learning Models, and Hyperparameters
#### Lluis Pellicer Juan y Jorge De la Cruz Martínez

**ACTIVITY**

Students must train and validate machine learning models for the "Oppositional thinking analysis: Conspiracy theories vs. critical thinking narratives" task. The goal is to distinguish conspiracy narratives from oppositional narratives that do not express a conspiracy mentality (i.e., critical thinking), a binary classification problem. For that purpose:
1. Students must train and evaluate at least four classifiers and compare their results. One of the classifiers must be an ensemble of three simple models.
2. For each of the three groups of text representation studied in the previous practice (traditional forms, static embedding-based, and contextual embedding-based), students have to choose one representation method and train the selected models using that representation. Preprocess the text as deemed appropriate for each representation method.
3. Students must train and evaluate models for both Spanish and English datasets.

The proposed models performance must be evaluated based on the official evaluation metric (Matthew correlation coefficient) provided by the task organizers. The official training dataset for both languages are in the file Dataset-Oppositional.zip

In [ ]:
import json
from nltk.corpus import stopwords
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
import torch
from transformers import BertTokenizer, BertModel
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

## Load data

In [ ]:
# Ruta de la carpeta donde se encuentran los archivos
carpeta_datos = "C:/Users/lluis/Desktop/lluis/Onedrive/Escritorio/3r Curso/2ndo cuatri/LNR/Dataset-Oppositionl"

# Ruta de los archivos en la carpeta de training
ruta_entrenamiento_en = os.path.join(carpeta_datos, "training", "dataset_oppositional", "dataset_en_train.json")
ruta_entrenamiento_es = os.path.join(carpeta_datos, "training", "dataset_oppositional", "dataset_es_train.json")

# Ruta de los archivos en la carpeta de test
ruta_test_en = os.path.join(carpeta_datos, "test", "dataset_oppositional_test_nolabels", "dataset_en_official_test_nolabels.json")
ruta_test_es = os.path.join(carpeta_datos, "test", "dataset_oppositional_test_nolabels", "dataset_es_official_test_nolabels.json")

# Función para cargar los datos de un archivo JSON
def cargar_datos(ruta):
    with open(ruta, "r", encoding="utf-8") as archivo:
        datos = json.load(archivo)
    return datos

# Cargar datos de los archivos
datos_entrenamiento_en = cargar_datos(ruta_entrenamiento_en)
datos_entrenamiento_es = cargar_datos(ruta_entrenamiento_es)
datos_test_en = cargar_datos(ruta_test_en)
datos_test_es = cargar_datos(ruta_test_es)

# Extraer textos y etiquetas
texts_es_train = [dato["text"] for dato in datos_entrenamiento_es]
labels_es_train = [dato["category"] for dato in datos_entrenamiento_es]

texts_en_train = [dato["text"] for dato in datos_entrenamiento_en]
labels_en_train = [dato["category"] for dato in datos_entrenamiento_en]

texts_es_test = [dato["text"] for dato in datos_test_es]
texts_en_test = [dato["text"] for dato in datos_test_en]

## Representation 1: Traditional forms -> CountVectorizer

### Preprocess and representation

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [ ]:
# Preprocesar textos en inglés
preprocessed_texts_en_train = [preprocess_text(text) for text in texts_en_train]
preprocessed_texts_en_test = [preprocess_text(text) for text in texts_en_test]

# Procesar textos preprocesados usando CountVectorizer para inglés
vectorizer_en = CountVectorizer(analyzer='word', max_features=4000, lowercase=True)
X_en_train = vectorizer_en.fit_transform(preprocessed_texts_en_train)
X_en_test = vectorizer_en.transform(preprocessed_texts_en_test)

# Convertir las matrices dispersas a matrices densas
X_en_train = X_en_train.toarray()
X_en_test = X_en_test.toarray()

# Convertir etiquetas a matrices numpy
Y_en_train = np.array(labels_en_train)

In [ ]:
# Preprocesar textos en español
preprocessed_texts_es_train = [preprocess_text(text) for text in texts_es_train]
preprocessed_texts_es_test = [preprocess_text(text) for text in texts_es_test]

# Procesar textos preprocesados usando CountVectorizer para español
vectorizer_es = CountVectorizer(analyzer='word', max_features=4000, lowercase=True)
X_es_train = vectorizer_es.fit_transform(preprocessed_texts_es_train)
X_es_test = vectorizer_es.transform(preprocessed_texts_es_test)

# Convertir las matrices dispersas a matrices densas
X_es_train = X_es_train.toarray()
X_es_test = X_es_test.toarray()

# Convertir etiquetas a matrices numpy
Y_es_train = np.array(labels_es_train)

### Train and evaluation of models

### SVC

In [ ]:
# Define los hiperparámetros que deseas ajustar -> predeterminados son 1 y rbf
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100]
}

# Para el conjunto de datos en inglés
X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(X_en_train, Y_en_train, test_size=0.1, random_state=1234)

# Inicializa el clasificador SVC
clf_en = svm.SVC()

# Realiza la búsqueda de cuadrícula
grid_search_en = GridSearchCV(estimator=clf_en, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search_en.fit(X_train_en, y_train_en)

# Muestra los mejores hiperparámetros encontrados
print("Best Parameters for English:")
print(grid_search_en.best_params_)

# Entrena el modelo con los mejores hiperparámetros encontrados
clf_en_best = grid_search_en.best_estimator_
clf_en_best.fit(X_train_en, y_train_en)

# Realiza predicciones en el conjunto de prueba
predicted_en = clf_en_best.predict(X_test_en)
quality_en = matthews_corrcoef(y_test_en, predicted_en)
print("MCC for English:", quality_en)

# Para el conjunto de datos en español
X_train_es, X_test_es, y_train_es, y_test_es = train_test_split(X_es_train, Y_es_train, test_size=0.1, random_state=1234)

# Inicializa el clasificador SVC
clf_es = svm.SVC()

# Realiza la búsqueda de cuadrícula
grid_search_es = GridSearchCV(estimator=clf_es, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search_es.fit(X_train_es, y_train_es)

# Muestra los mejores hiperparámetros encontrados
print("Best Parameters for Spanish:")
print(grid_search_es.best_params_)

# Entrena el modelo con los mejores hiperparámetros encontrados
clf_es_best = grid_search_es.best_estimator_
clf_es_best.fit(X_train_es, y_train_es)

# Realiza predicciones en el conjunto de prueba
predicted_es = clf_es_best.predict(X_test_es)
quality_es = matthews_corrcoef(y_test_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Logistic Regression

In [ ]:
# Definir los hiperparámetros a probar
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
}

# Para el conjunto de datos en inglés
grid_search_en = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy')
grid_search_en.fit(X_train_en, y_train_en)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para inglés:", grid_search_en.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para inglés
clf_en = grid_search_en.best_estimator_
clf_en.fit(X_train_en, y_train_en)
predicted_en = clf_en.predict(X_test_en)
quality_en = matthews_corrcoef(y_test_en, predicted_en)
print("MCC for English:", quality_en)
print()

# Para el conjunto de datos en español
grid_search_es = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy')
grid_search_es.fit(X_train_es, y_train_es)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para español:", grid_search_es.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para español
clf_es = grid_search_es.best_estimator_
clf_es.fit(X_train_es, y_train_es)
predicted_es = clf_es.predict(X_test_es)
quality_es = matthews_corrcoef(y_test_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Decision Trees

In [ ]:
# Definir los hiperparámetros a probar
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Para el conjunto de datos en inglés
grid_search_en = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
grid_search_en.fit(X_train_en, y_train_en)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para inglés:", grid_search_en.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para inglés
clf_en = grid_search_en.best_estimator_
clf_en.fit(X_train_en, y_train_en)
predicted_en = clf_en.predict(X_test_en)
quality_en = matthews_corrcoef(y_test_en, predicted_en)
print("MCC for English:", quality_en)
print()

# Para el conjunto de datos en español
grid_search_es = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
grid_search_es.fit(X_train_es, y_train_es)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para español:", grid_search_es.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para español
clf_es = grid_search_es.best_estimator_
clf_es.fit(X_train_es, y_train_es)
predicted_es = clf_es.predict(X_test_es)
quality_es = matthews_corrcoef(y_test_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Ensemble model: Stacking

In [ ]:
# Mejores parámetros para SVC
svc_params_en = {'C': 0.1, 'kernel': 'linear'}
svc_params_es = {'C': 10, 'kernel': 'rbf'}

# Mejores parámetros para Logistic Regression
lr_params_en = {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
lr_params_es = {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}

# Mejores parámetros para Decision Trees
dt_params_en = {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
dt_params_es = {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'splitter': 'random'}

# Crear modelos base para el clasificador de apilamiento
base_models_en = [('svc', SVC(**svc_params_en)), ('lr', LogisticRegression(**lr_params_en)), ('dt', DecisionTreeClassifier(**dt_params_en))]
base_models_es = [('svc', SVC(**svc_params_es)), ('lr', LogisticRegression(**lr_params_es)), ('dt', DecisionTreeClassifier(**dt_params_es))]

# Crear el meta-modelo
meta_model_en = LogisticRegression()
meta_model_es = LogisticRegression()

# Crear el clasificador de apilamiento
ensemble_en = StackingClassifier(estimators=base_models_en, final_estimator=meta_model_en)
ensemble_es = StackingClassifier(estimators=base_models_es, final_estimator=meta_model_es)

# Entrenar el clasificador de apilamiento en los datos de entrenamiento
ensemble_en.fit(X_train_en, y_train_en)
ensemble_es.fit(X_train_es, y_train_es)

# Evaluar el clasificador de apilamiento en los datos de prueba
predicted_en = ensemble_en.predict(X_test_en)
predicted_es = ensemble_es.predict(X_test_es)

# Calcular la calidad de la predicción usando MCC
quality_en = matthews_corrcoef(y_test_en, predicted_en)
quality_es = matthews_corrcoef(y_test_es, predicted_es)

print("MCC for English:", quality_en)
print("MCC for Spanish:", quality_es)

## Representation 2: Static Embedding-Based -> Word2vec

###  Preprocess and representation

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [ ]:
# Preprocesar textos en inglés
preprocessed_texts_en_train = [preprocess_text(text) for text in texts_en_train]
preprocessed_texts_en_test = [preprocess_text(text) for text in texts_en_test]

# Combinar todos los textos en una sola lista para entrenar el modelo
all_texts_en = preprocessed_texts_en_train + preprocessed_texts_en_test

# Tokenizar los textos
tokenized_en = [word_tokenize(text.lower()) for text in all_texts_en]

# Entrenar el modelo Word2Vec en inglés
word2vec_model_en = Word2Vec(sentences=tokenized_en, vector_size=300, window=5, min_count=1, workers=4)

# Función para obtener las incrustaciones de Word2Vec
def get_word2vec_embeddings_en(text):
    tokens = word_tokenize(text.lower())
    embeddings = []
    for token in tokens:
        try:
            embedding = word2vec_model_en.wv[token]
            embeddings.append(embedding)
        except KeyError:
            pass
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return None

# Aplicar Word2Vec a los textos en inglés
X_word2vec_train_en = np.array([get_word2vec_embeddings_en(text) for text in preprocessed_texts_en_train])
X_word2vec_test_en = np.array([get_word2vec_embeddings_en(text) for text in preprocessed_texts_en_test])
Y_en_train = np.array(labels_en_train)

In [ ]:
# Preprocesar textos en español
preprocessed_texts_es_train = [preprocess_text(text) for text in texts_es_train]
preprocessed_texts_es_test = [preprocess_text(text) for text in texts_es_test]

# Combinar todos los textos en una sola lista para entrenar el modelo
all_texts_es = preprocessed_texts_es_train + preprocessed_texts_es_test

# Tokenizar los textos
tokenized_es = [word_tokenize(text.lower()) for text in all_texts_es]

# Entrenar el modelo Word2Vec en español
word2vec_model_es = Word2Vec(sentences=tokenized_es, vector_size=300, window=5, min_count=1, workers=4)

# Función para obtener las incrustaciones de Word2Vec
def get_word2vec_embeddings_es(text):
    tokens = word_tokenize(text.lower())
    embeddings = []
    for token in tokens:
        try:
            embedding = word2vec_model_es.wv[token]
            embeddings.append(embedding)
        except KeyError:
            pass
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return None

# Aplicar Word2Vec a los textos en español
X_word2vec_train_es = np.array([get_word2vec_embeddings_es(text) for text in preprocessed_texts_es_train])
X_word2vec_test_es = np.array([get_word2vec_embeddings_es(text) for text in preprocessed_texts_es_test])
Y_es_train = np.array(labels_es_train)

### Train and evaluation of models

### SVC

In [ ]:
# Definir los parámetros a probar
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100]
}

# Para el conjunto de datos en inglés
X_train_word2vec_en, X_test_word2vec_en, y_train_word2vec_en, y_test_word2vec_en = train_test_split(X_word2vec_train_en, Y_en_train, test_size=0.1, random_state=1234)

# Inicializa el clasificador SVC
clf_en = svm.SVC()

# Realiza la búsqueda de cuadrícula
grid_search_en = GridSearchCV(estimator=clf_en, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search_en.fit(X_train_word2vec_en, y_train_word2vec_en)

# Muestra los mejores hiperparámetros encontrados
print("Best Parameters for English:")
print(grid_search_en.best_params_)

# Entrena el modelo con los mejores hiperparámetros encontrados
clf_en_best = grid_search_en.best_estimator_
clf_en_best.fit(X_train_word2vec_en, y_train_word2vec_en)

# Realiza predicciones en el conjunto de prueba
predicted_word2vec_en = clf_en_best.predict(X_test_word2vec_en)
quality_en = matthews_corrcoef(y_test_word2vec_en, predicted_word2vec_en)
print("MCC for English:", quality_en)


# Para el conjunto de datos en español
X_train_word2vec_es, X_test_word2vec_es, y_train_word2vec_es, y_test_word2vec_es = train_test_split(X_word2vec_train_es, Y_es_train, test_size=0.1, random_state=1234)

# Inicializa el clasificador SVC
clf_es = svm.SVC()

# Realiza la búsqueda de cuadrícula
grid_search_es = GridSearchCV(estimator=clf_es, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search_es.fit(X_train_word2vec_es, y_train_word2vec_es)

# Muestra los mejores hiperparámetros encontrados
print("Best Parameters for Spanish:")
print(grid_search_es.best_params_)

# Entrena el modelo con los mejores hiperparámetros encontrados
clf_es_best = grid_search_es.best_estimator_
clf_es_best.fit(X_train_word2vec_es, y_train_word2vec_es)

# Realiza predicciones en el conjunto de prueba
predicted_word2vec_es = clf_es_best.predict(X_test_word2vec_es)
quality_es = matthews_corrcoef(y_test_word2vec_es, predicted_word2vec_es)
print("MCC for Spanish:", quality_es)

### Logistic regression

In [ ]:
# Definir los hiperparámetros a probar
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
}

# Para el conjunto de datos en inglés
grid_search_en = GridSearchCV(LogisticRegression(), param_grid, cv=2, scoring='accuracy')
grid_search_en.fit(X_train_word2vec_en, y_train_word2vec_en)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para inglés:", grid_search_en.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para inglés
clf_en = grid_search_en.best_estimator_
clf_en.fit(X_train_word2vec_en, y_train_word2vec_en)
predicted_en = clf_en.predict(X_test_word2vec_en)
quality_en = matthews_corrcoef(y_test_word2vec_en, predicted_en)
print("MCC for English:", quality_en)
print()

# Para el conjunto de datos en español
grid_search_es = GridSearchCV(LogisticRegression(), param_grid, cv=2, scoring='accuracy')
grid_search_es.fit(X_train_word2vec_es, y_train_word2vec_es)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para español:", grid_search_es.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para español
clf_es = grid_search_es.best_estimator_
clf_es.fit(X_train_word2vec_es, y_train_word2vec_es)
predicted_es = clf_es.predict(X_test_word2vec_es)
quality_es = matthews_corrcoef(y_test_word2vec_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Decision trees

In [ ]:
# Definir los hiperparámetros a probar
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Para el conjunto de datos en inglés
grid_search_en = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
grid_search_en.fit(X_train_word2vec_en, y_train_word2vec_en)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para inglés:", grid_search_en.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para inglés
clf_en = grid_search_en.best_estimator_
clf_en.fit(X_train_word2vec_en, y_train_word2vec_en)
predicted_en = clf_en.predict(X_test_word2vec_en)
quality_en = matthews_corrcoef(y_test_word2vec_en, predicted_en)
print("MCC for English:", quality_en)
print()

# Para el conjunto de datos en español
grid_search_es = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
grid_search_es.fit(X_train_word2vec_es, y_train_word2vec_es)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para español:", grid_search_es.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para español
clf_es = grid_search_es.best_estimator_
clf_es.fit(X_train_word2vec_es, y_train_word2vec_es)
predicted_es = clf_es.predict(X_test_word2vec_es)
quality_es = matthews_corrcoef(y_test_word2vec_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Stacking

In [ ]:
# Mejores parámetros para SVC
svc_params_en = {'C': 100, 'kernel': 'linear'}
svc_params_es = {'C': 100, 'kernel': 'linear'}

# Mejores parámetros para Logistic Regression
lr_params_en = {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
lr_params_es = {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}

# Mejores parámetros para Decision Trees
dt_params_en = {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'random'}
dt_params_es = {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}

# Crear modelos base para el clasificador de apilamiento
base_models_en = [('svc', SVC(**svc_params_en)), ('lr', LogisticRegression(**lr_params_en)), ('dt', DecisionTreeClassifier(**dt_params_en))]
base_models_es = [('svc', SVC(**svc_params_es)), ('lr', LogisticRegression(**lr_params_es)), ('dt', DecisionTreeClassifier(**dt_params_es))]

# Crear el meta-modelo
meta_model_en = LogisticRegression()
meta_model_es = LogisticRegression()

# Crear el clasificador de apilamiento
ensemble_en = StackingClassifier(estimators=base_models_en, final_estimator=meta_model_en)
ensemble_es = StackingClassifier(estimators=base_models_es, final_estimator=meta_model_es)

# Entrenar el clasificador de apilamiento en los datos de entrenamiento
ensemble_en.fit(X_train_word2vec_en, y_train_word2vec_en)
ensemble_es.fit(X_train_word2vec_es, y_train_word2vec_es)

# Evaluar el clasificador de apilamiento en los datos de prueba
predicted_en = ensemble_en.predict(X_test_word2vec_en)
predicted_es = ensemble_es.predict(X_test_word2vec_es)

# Calcular la calidad de la predicción usando MCC
quality_en = matthews_corrcoef(y_test_word2vec_en, predicted_en)
quality_es = matthews_corrcoef(y_test_word2vec_es, predicted_es)

print("MCC for English:", quality_en)
print("MCC for Spanish:", quality_es)

## Representation 3: Contextual Embedding-Based -> BertModel

### Representation

In [ ]:
# Cargar el tokenizador y el modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Función para obtener las incrustaciones contextuales con BERT
def get_bert_embeddings(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token embeddings
    return cls_embeddings.numpy()

# Aplicar BERT a los textos en inglés
X_train_bert_en = np.array([get_bert_embeddings(text) for text in texts_en_train])
# Aplicar BERT a los textos en español
X_train_bert_es = np.array([get_bert_embeddings(text) for text in texts_es_train])

In [ ]:
# Para el conjunto de datos en inglés con incrustaciones BERT
X_train_bert_en, X_test_bert_en, y_train_bert_en, y_test_bert_en = train_test_split(X_train_bert_en, labels_en_train, test_size=0.1, random_state=1234)

In [ ]:
# Para el conjunto de datos en inglés con incrustaciones BERT
X_train_bert_es, X_test_bert_es, y_train_bert_es, y_test_bert_es = train_test_split(X_train_bert_es, labels_es_train, test_size=0.1, random_state=1234)

### Train and Evaluation of models

### SVC

In [ ]:
# Definir los parámetros a probar
param_grid_svc = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly']
}

X_train_bert_en_flat = X_train_bert_en.reshape(X_train_bert_en.shape[0], -1)
X_train_bert_es_flat = X_train_bert_es.reshape(X_train_bert_es.shape[0], -1)

# Para el conjunto de datos en inglés
grid_search_svc_en = GridSearchCV(SVC(), param_grid_svc, cv=3, scoring='accuracy')
grid_search_svc_en.fit(X_train_bert_en_flat, y_train_bert_en)

# Imprimir los mejores hiperparámetros encontrados para inglés
print("Mejores hiperparámetros para inglés:", grid_search_svc_en.best_params_)

# Mejores parámetros para SVC en inglés
best_params_svc_en = grid_search_svc_en.best_params_

# Inicializar y ajustar el modelo SVC en inglés con los mejores hiperparámetros
svc_en = SVC(**best_params_svc_en)
svc_en.fit(X_train_bert_en_flat, y_train_bert_en)

# Aplanar los datos de prueba en inglés
X_test_bert_en_flat = X_test_bert_en.reshape(X_test_bert_en.shape[0], -1)

# Evaluar el modelo en inglés
predicted_en = svc_en.predict(X_test_bert_en_flat)
quality_en = matthews_corrcoef(y_test_bert_en, predicted_en)
print("MCC for English:", quality_en)

# Para el conjunto de datos en español
grid_search_svc_es = GridSearchCV(SVC(), param_grid_svc, cv=3, scoring='accuracy')
grid_search_svc_es.fit(X_train_bert_es_flat, y_train_bert_es)

# Imprimir los mejores hiperparámetros encontrados para español
print("Mejores hiperparámetros para español:", grid_search_svc_es.best_params_)

# Mejores parámetros para SVC en español
best_params_svc_es = grid_search_svc_es.best_params_

# Inicializar y ajustar el modelo SVC en español con los mejores hiperparámetros
svc_es = SVC(**best_params_svc_es)
svc_es.fit(X_train_bert_es_flat, y_train_bert_es)

# Aplanar los datos de prueba en español
X_test_bert_es_flat = X_test_bert_es.reshape(X_test_bert_es.shape[0], -1)

# Evaluar el modelo en español
predicted_es = svc_es.predict(X_test_bert_es_flat)
quality_es = matthews_corrcoef(y_test_bert_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Logistic Regression

In [ ]:
# Definir los hiperparámetros a probar
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
}

X_train_bert_en_flat = X_train_bert_en.reshape(X_train_bert_en.shape[0], -1)
X_train_bert_es_flat = X_train_bert_es.reshape(X_train_bert_es.shape[0], -1)

# Para el conjunto de datos en inglés con incrustaciones BERT
grid_search_en = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy')
grid_search_en.fit(X_train_bert_en_flat, y_train_bert_en)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para inglés:", grid_search_en.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para inglés
clf_en = grid_search_en.best_estimator_
clf_en.fit(X_train_bert_en_flat, y_train_bert_en)

# Aplanar los datos de prueba en inglés
X_test_bert_en_flat = X_test_bert_en.reshape(X_test_bert_en.shape[0], -1)

predicted_en = clf_en.predict(X_test_bert_en_flat)
quality_en = matthews_corrcoef(y_test_bert_en, predicted_en)
print("MCC for English:", quality_en)
print()

# Para el conjunto de datos en español con incrustaciones BERT
grid_search_es = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy')
grid_search_es.fit(X_train_bert_es_flat, y_train_bert_es)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para español:", grid_search_es.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para español
clf_es = grid_search_es.best_estimator_
clf_es.fit(X_train_bert_es_flat, y_train_bert_es)
X_test_bert_es_flat = X_test_bert_es.reshape(X_test_bert_es.shape[0], -1)
predicted_es = clf_es.predict(X_test_bert_es_flat)
quality_es = matthews_corrcoef(y_test_bert_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Decision Trees

In [ ]:
# Definir los hiperparámetros a probar
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

X_train_bert_en_flat = X_train_bert_en.reshape(X_train_bert_en.shape[0], -1)
X_train_bert_es_flat = X_train_bert_es.reshape(X_train_bert_es.shape[0], -1)

# Para el conjunto de datos en inglés con incrustaciones BERT
grid_search_en = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
grid_search_en.fit(X_train_bert_en_flat, y_train_bert_en)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para inglés:", grid_search_en.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para inglés
clf_en = grid_search_en.best_estimator_
clf_en.fit(X_train_bert_en_flat, y_train_bert_en)

# Aplanar los datos de prueba en inglés
X_test_bert_en_flat = X_test_bert_en.reshape(X_test_bert_en.shape[0], -1)

predicted_en = clf_en.predict(X_test_bert_en_flat)
quality_en = matthews_corrcoef(y_test_bert_en, predicted_en)
print("MCC for English:", quality_en)
print()

# Para el conjunto de datos en español con incrustaciones BERT
grid_search_es = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, scoring='accuracy')
grid_search_es.fit(X_train_bert_es_flat, y_train_bert_es)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para español:", grid_search_es.best_params_)
print()

# Entrenar el modelo con los mejores hiperparámetros para español
clf_es = grid_search_es.best_estimator_
clf_es.fit(X_train_bert_es_flat, y_train_bert_es)
X_test_bert_es_flat = X_test_bert_es.reshape(X_test_bert_es.shape[0], -1)
predicted_es = clf_es.predict(X_test_bert_es_flat)
quality_es = matthews_corrcoef(y_test_bert_es, predicted_es)
print("MCC for Spanish:", quality_es)

### Stacking

In [ ]:
# Mejores parámetros para SVC
svc_params_en = {'C': 10, 'kernel': 'rbf'}
svc_params_es = {'C': 10, 'kernel': 'rbf'}

# Mejores parámetros para Logistic Regression
lr_params_en = {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
lr_params_es = {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}

# Mejores parámetros para Decision Trees
dt_params_en = {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
dt_params_es = {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'splitter': 'best'}

# Crear modelos base para el clasificador de apilamiento
base_models_en = [('svc', SVC(**svc_params_en)), ('lr', LogisticRegression(**lr_params_en)), ('dt', DecisionTreeClassifier(**dt_params_en))]
base_models_es = [('svc', SVC(**svc_params_es)), ('lr', LogisticRegression(**lr_params_es)), ('dt', DecisionTreeClassifier(**dt_params_es))]

# Crear el meta-modelo
meta_model_en = LogisticRegression()
meta_model_es = LogisticRegression()

X_train_bert_en_flat = X_train_bert_en.reshape(X_train_bert_en.shape[0], -1)
X_train_bert_es_flat = X_train_bert_es.reshape(X_train_bert_es.shape[0], -1)
X_test_bert_en_flat = X_test_bert_en.reshape(X_test_bert_en.shape[0], -1)
X_test_bert_es_flat = X_test_bert_es.reshape(X_test_bert_es.shape[0], -1)

# Crear el clasificador de apilamiento
ensemble_en = StackingClassifier(estimators=base_models_en, final_estimator=meta_model_en)
ensemble_es = StackingClassifier(estimators=base_models_es, final_estimator=meta_model_es)

# Entrenar el clasificador de apilamiento en los datos de entrenamiento
ensemble_en.fit(X_train_bert_en_flat, y_train_bert_en)
ensemble_es.fit(X_train_bert_es_flat, y_train_bert_es)

# Evaluar el clasificador de apilamiento en los datos de prueba
predicted_en = ensemble_en.predict(X_test_bert_en_flat)
predicted_es = ensemble_es.predict(X_test_bert_es_flat)

# Calcular la calidad de la predicción usando MCC
quality_en = matthews_corrcoef(y_test_bert_en, predicted_en)
quality_es = matthews_corrcoef(y_test_bert_es, predicted_es)

print("MCC for English:", quality_en)
print("MCC for Spanish:", quality_es)